In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from shutil import copy2 as copy
from IPython.display import display, HTML
from math import floor
import json
import re
import os

secrets = json.load(open('secrets.json'))
game_version = secrets['game_version']
os.makedirs(game_version, exist_ok=True)

# Awakening "Player" Classes

In [ ]:
classes_meta = pd.read_csv(f'data/classes_meta.csv', sep='\s*?,\s*?', engine='python').fillna('')
core_maxes = pd.read_csv('data/classes_core.csv')
rank_stats = ['Str', "Mag", 'Skl', 'Spd', 'Def', 'Res']
stats = ['Str', "Mag", 'Skl', 'Spd', 'Def', 'Res']

core_maxes = core_maxes.merge(classes_meta.drop(rank_stats, axis=1), on='Class')
def attack_stat(x):
    if re.search('Reason|Faith|Dark|Staves', x):
        if re.search('Swords|Bows|Lances|Axes|Stones', x):
            return "Dual"
        else: return 'Mag'
    else: return 'Phys'
core_maxes['Atk'] = core_maxes.Wield.apply(lambda x: attack_stat(x))

class_tiers = core_maxes.loc[core_maxes['Rank']=='Advanced', 'Class'].to_frame()
for x in stats:
    class_tiers[x] = pd.cut(core_maxes.loc[core_maxes['Rank']=='Advanced',x], 5, retbins=True, labels=['E', 'D', 'C', 'B', 'A'])[0]
class_data = class_tiers.merge(core_maxes[['Class', 'Atk', 'Wield', 'Rank', 'Tags']], on='Class')

# # Dashes Str/Mag if the unit can't use a weapon of that type, Averages Dual type value to both.
# def _func(x):
#     if x.Atk == 'Mag': x['Str'] = '-'
#     if x.Atk == 'Phys': x['Mag'] = '-'
#     if x.Atk == 'Dual': 
#         x1 = {'E':1, 'D':2, 'C':3, 'B':4, 'A':5}[x['Str']]
#         x2 = {'E':1, 'D':2, 'C':3, 'B':4, 'A':5}[x['Mag']]
#         x['Str'] = {1:'E', 2:'D', 3:'C', 4:'B', 5:'A'}[round((x1+x2-1)/2)]
#         x['Mag'] = {1:'E', 2:'D', 3:'C', 4:'B', 5:'A'}[round((x1+x2-1)/2)]
#     return x
# class_data = class_data.apply(lambda x: _func(x), axis=1).drop(['Atk'], axis=1)

class_tiers['Total'] = class_tiers[stats].apply(lambda x: x.map({'-':0, 'E':1, 'D':2, 'C':3, 'B':4, 'A':5})).sum(axis=1)
class_tiers.sort_values('Total')
# same_stats = class_tiers.groupby(rank_stats)['Class'].apply(','.join).reset_index()
# same_stats = same_stats[same_stats.Class.notnull()]
# same_stats['Count'] = same_stats.Class.str.count(',')+1
# same_stats

In [ ]:
export = class_tiers.copy()
export["Atk"] = export.apply(lambda x: x.Mag if x.Mag != '-' else x.Str, axis=1)
export['HP'] = 20
export_order = ['Class', 'HP', 'Str', 'Mag', 'Skl', 'Spd', 'Def', 'Res', 'Wield', 'Tags', 'Rank']
export = export[export_order].sort_values('Class')
md_table = export.to_markdown(index=0)

export.to_csv(f'{game_version}/classes.csv', index=False)
open(f'{game_version}/Classes.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Classes.md', 'w').write(md_table)
print(md_table)

# Engage Classes

In [2]:
engage_raw = pd.read_csv('data/engage_classes.csv', sep='\s*,\s*', engine='python')
engage_raw['Tags'] = engage_raw['Tags'].fillna('')
stats = ['HP', 'Str', "Mag", 'Dex', 'Spd', 'Def', 'Res']

# Pre rating tuning

for col in ['Def']:
    engage_raw[col] = engage_raw[col].apply(lambda x: x-12 if x > 50 else x)

# Rating generation

class_data = engage_raw.loc[(~engage_raw.Unique), ['Class', 'Weapons']]
for x in stats:
    class_data[x] = pd.cut(engage_raw.loc[(~engage_raw.Unique),x], 5, retbins=True, 
labels=['E', 'D', 'C', 'B', 'A'])[0]
class_data = class_data.merge(engage_raw, on=['Class', 'Weapons'], suffixes=[None, 'Value'])


# Export cleansing and tuning

export = class_data.copy()
export["Tags"] = ((export.Tags + ' ' + export.Weak + ' '+ export.Type).str.strip().str.split(' ')).apply(lambda x: ', '.join(set([y for y in x if y != '—'])))
export["Tags"] = export.apply(lambda x: x.Tags if x.Bld < 16 else x.Tags+', Built', axis=1)
export["Tags"] = export.Tags.str.replace('Qi,', 'Qi')
export['Promoted'] = export.Promoted.map({True:"Yes", False:'No'})

for x in stats: export[x] = export[x].astype('str')
grooms = pd.read_csv('data/engage_class_groom.csv', sep='\s*,\s*', engine='python')
for i, x in grooms.iterrows():
    export.loc[export.Class.str.contains(x.Class), stats] = list(x.loc[stats])

base_hp = 25
HP_factor = 2
export['Total'] = export[stats].apply(lambda x: x.map({'-':0, 'E':1, 'D':2, 'C':3, 'B':4, 
'A':5})).sum(axis=1)
export['HP'] = export['HP'].map({'E':base_hp-(2*HP_factor), 'D':base_hp-(HP_factor), 'C':base_hp, 'B':base_hp+HP_factor, 'A':base_hp+(2*HP_factor)})

export_order = ['Class', 'HP', 'Str', 'Mag', 'Dex', 'Spd', 'Def', 'Res', 'Proficencies', 'Tags', 'Promoted', 'Total']
export = export.rename({'Weapons':"Proficencies"}, axis=1)
export = export[export_order].sort_values(['Promoted','Class'])
md_table = export.to_markdown(index=0)

same_stats = export.groupby(stats)['Class'].apply(','.join).reset_index()
same_stats = same_stats[export.Class.notnull()]
same_stats['Count'] = same_stats.Class.str.count(',')+1

# ---

# class_data
# print(class_data.sort_values(['Def']).reset_index(drop=True))
print((export[stats].agg(pd.value_counts)/52).round(2))
# same_stats
# export#[export.Promoted == 'No']


      HP   Str   Mag   Dex   Spd   Def   Res
21  0.06   NaN   NaN   NaN   NaN   NaN   NaN
23  0.13   NaN   NaN   NaN   NaN   NaN   NaN
25  0.42   NaN   NaN   NaN   NaN   NaN   NaN
27  0.29   NaN   NaN   NaN   NaN   NaN   NaN
29  0.10   NaN   NaN   NaN   NaN   NaN   NaN
A    NaN  0.17  0.04  0.10  0.06  0.17  0.04
B    NaN  0.29  0.10  0.33  0.17  0.17  0.17
C    NaN  0.46  0.04  0.50  0.50  0.35  0.29
D    NaN  0.04  0.29  0.08  0.21  0.23  0.35
E    NaN  0.04  0.54   NaN  0.06  0.08  0.15


/var/folders/ks/dksml1ms0rg9dk9rjyqpnfdc0000gn/T/ipykernel_2694/3348787622.py:44: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  same_stats = same_stats[export.Class.notnull()]


In [5]:
os.makedirs(f'{secrets["ObsidianRoot"]}/{game_version}', exist_ok=True)
export.to_csv(f'{game_version}/classes_engage.csv', index=False)
open(f'{game_version}/Classes.md', 'w').write(md_table)
open(f'{secrets["ObsidianRoot"]}/{game_version}/Classes.md', 'w').write(md_table)
# print(md_table)

8099

In [ ]:
dam, defn = 4, 0
df = pd.DataFrame()
for x, r in zip(range(dam, dam+8), 'FEDCBAS'):
    for x2, r2 in zip(range(defn, defn+8), 'FEDCBAS'):
        df.loc[r, r2] = x-x2
df = df.astype('int').clip(lower=0, upper=8)